# Idea 2: generic games

A Python object is orderable if it provides a 'less than' operator. Armed with this, an algorithm can sort a collection of objects without knowing anything more about them.


In the same vein, let's say that a game state is something that:

* Has a current player
* Has a current result - one of 'won/lost/drawn/in progress'
* Can provide a list of legal commands
* Can produce a new state when given a legal command

```python
class Result(str, Enum):
    PLAYER1 = "PLAYER 1 WINS"
    PLAYER2 = "PLAYER 2 WINS"
    DRAW = "DRAW"
    INPROGRESS = "IN PROGRESS"


class Player(str, Enum):
    ONE = "ONE"
    TWO = "TWO"

    
class StateProtocol(Protocol):
    def apply(self, command: Any) -> "StateProtocol":
        ...

    player: Player
    result: Result
    commands: List

```

Any number of popular board games can be described with this model: Chess, Go, Connect-4, tic-tac-toe etc.

Armed with this, we could create an algorithm that could play any of these games perfectly, from any position:

* For a given state, for every possible command, call `apply` and get a child state
* Repeat step 1 for each of those states
* Keep repeating until you've built out a tree of every possible game position reachable from this state
* Choose the move that leads to the highest win/loss ratio.


This works fine for very small games, but rapidly becomes untenable for games with many possible moves.


Chess probably has more than $10^{50}$ possible game states, and Go has a game tree complexity of $10^{360}$

So we need a way of reducing drastically how many positions we actually look at.

One approach is minimax trees and certain improvements such as alpha-beta pruning. These work by only navigating a certain depth down the tree, but that introduces two problems

* Performance - adding another layer of depth can make the algorithm take N times longer to run, where N is the typical number of moves available.
* Evaluation - it only works if you have SOME way of evaluating/scoring a given position. We're interested in an algorithm that knows *nothing* about the game in question.